# CY Orientifolds

This notebook provides instructions on how to read the data provided in our GitHub repository for Calabi-Yau (CY) orientifolds. 

Author: Andreas Schachner

E-Mail: andreas.schachner@gmx.net


## Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import seaborn as sn
sn.set(rc={"figure.dpi":200, 'savefig.dpi':200})
sn.set_context('notebook')
sn.set_style("ticks")
import sys
import glob


import pickle
import gzip
        
def load_zipped_pickle(filen):
    
    with gzip.open(filen, 'rb') as f:
        loaded_object = pickle.load(f)
        
    f.close()
        
    return loaded_object


# Read data

To read the data, we pick a value for $h^{1,1}$ and choose the type of data:

* **Complete**: Contains complete data for *all* FRSTs of *all* favourable polytopes at $2\leq h^{1,1}\leq 7$
* **Random**: Contains data obtained from random sampling at $7\leq h^{1,1}\leq 12$

Below, we work with the **Complete** data by setting **random_data=0** below.

In [21]:
h11=2

random_data=0

file="POLY_*"

home_dir=os.getcwd()

if random_data==0:
    filename = home_dir+"/Data/Complete/h11_"+str(h11)+"/"+file+".p"
else:
    filename = home_dir+"/Data/Random/h11_"+str(h11)+"/"+file+".p"
            
            
filenames =glob.glob(filename)
filenames[:5]

['/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_10_h11_2_h12_86.p',
 '/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_11_h11_2_h12_86.p',
 '/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_12_h11_2_h12_86.p',
 '/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_13_h11_2_h12_86.p',
 '/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_14_h11_2_h12_86.p']

The list **filenames** contains all filenames for polytopes with a given h11. If we want, we can further limit ourselves to polytopes with a specified h12:

In [22]:
h12=272
filenames=[file for file in filenames if "h12_"+str(h12) in file]
filenames

['/home/CY_Orientifold_database/Data/Complete/h11_2/POLY_36_h11_2_h12_272.p']

We picked $h^{1,2}=272$ which corresponds to the degree 18 hypersurface in $\mathbb{CP}[1,1,1,6,9]$ as discussed in Appendix B.1 in the paper. This leaves us with one filename which we now read in using the function **load_zipped_pickle**:

In [23]:
if len(filenames)>0:
    filename=filenames[0]
else:
    print("No filename found!")
    sys.exit()

    
ptr=load_zipped_pickle(filename)

We loaded the file into a dictionary **ptr** with the following keys:

In [24]:
ptr.keys()

dict_keys(['h11', 'h12', 'chi', 'PPOINTS', 'WEIGHTS', 'FIBRE', 'SIMPS', 'SRI', 'INTNUMS', 'SCHERN', 'DIVS', 'DPEZ', 'OFOLDS', 'STAT'])

The keys provide access to the following data:
    
| Key | Quantity | Type | Length |
| --- | --- | --- | --- |
| **'h11'** | $h^{1,1}(X_3)$ of the corresponding CY $X_3$ | scalar | 0 |
| **'h12'** | $h^{1,2}(X_3)$ of the corresponding CY $X_3$ | scalar | 0 |
| **'chi'** | $\chi(X_3)$ of the corresponding CY $X_3$ | scalar | 0 |
| **'PPOINTS'** | $N$ lattice points | array | h11+4 |
| **'WEIGHTS'** | GLSM weight matrix | array | h11+4 |
| **'FIBRE'** | Fiberation structre of the polytope | array | - |
| **'SIMPS'** | Simplices of all FRSTs | array | #FRSTs |
| **'SRI'** | SR-ideal for all FRSTs | array | #FRSTs |
| **'INTNUMS'** | Intersection numbers for all FRSTs | array | #FRSTs |
| **'SCHERN'** | 2nd Chern class for all FRSTs | array | #FRSTs |
| **'DIVS'** | Topological numbers of divisors for all FRSTs | array | #FRSTs |
| **'DPEZ'** | (Diagonal) del Pezzo divisors for all FRSTs | array | #FRSTs |
| **'OFOLDS'** | Orientifolds for all FRSTs | array | #FRSTs |
| **'STAT'** | Statistic for the polytope | dictionary | - |
    

The triangulation dependent arrays have length of the number of triangulations.

The intersection numbers include the anti-canonical divisor at index 0.

The additional array of del Pezzo divisors does not use the information about Hodge numbers. Indeed, in some cases, a divisor might look topologically like a del Pezzo, but there are actually obstructions related to the precise embedding into the CY which can e.g. be seen on the level of intersection numbers. In the list of del Pezzo divisors, we check the necessary conditions in terms of the intersection structures stated in our paper, namely for a del Pezzo $dP_n$ divisor $D_s$:

$\int_{X_3}\, D_s^3=\kappa_{sss}=9-n>0\qquad \int_{X_3}\, D_s^2 D_i\leq 0\quad \forall i\neq s$

Furthermore, we check the diagonality condition

$\kappa_{sss}\kappa_{sij}=\kappa_{ssi}\kappa_{ssj}\quad \forall i,j$

which is usually required either for applications to the LVS or models with branes at singularities. The array of divisors $D_{s_i}$ of del Pezzo type dP$_{n_i}$ associated with the key **'DPEZ'** in structured as:

$\text{ptr["DPEZ"]}=[[s_1, n_1,d_1],[s_2, n_2,d_2],\ldots ]$

Here, $d_i=1$ means that the divisor is diagonal, and $d_i=0$ that it is non-diagonal. We give one concrete example further below.


In [25]:
Orientifolds_tr=ptr["OFOLDS"]
Divisors_tr=ptr["DIVS"]

Let us specify a triangulation **T**:

In [26]:
T=0
Orientifolds=Orientifolds_tr[T]
Divisors=Divisors_tr[T]

Let us first look at the divisor data. The topological numbers read

In [27]:
Divisors

array([[1, list([1, 0, 2, 30]), 36, 144.0, 3.0],
       [2, list([1, 0, 2, 30]), 36, 144.0, 3.0],
       [3, list([1, 0, 2, 30]), 36, 144.0, 3.0],
       [4, list([1, 0, 28, 218]), 276, 5424.0, 29.0],
       [5, list([1, 0, 65, 417]), 549, 16776.0, 66.0],
       [6, list([1, 0, 0, 1]), 3, 552.0, 1.0]], dtype=object)

There are **five** columns which encode the following quantities

| Column | Quantity | Type | Length |
| --- | --- | --- | --- |
| 1 | Divisor/coordinate index $i$ for $D_i=\lbrace z_i=0\rbrace$ | scalar | 0 |
| 2 | Hodge numbers $h^{p,q}(D_i)$ in the format $[h^{0,0},h^{0,1}, h^{0,2},h^{1,1}]$ | list | 4 |
| 3 | Euler number $\chi(D_i)$ | scalar | 0 |
| 4 | Euler number $\chi(4D_i)$ | scalar | 0 |
| 5 | Arithmetic genus $\chi_0(D_i)$ | scalar | 0 |

As an example, let us take look at the divisor $D_6$

In [28]:
D=5
D6=Divisors[D]
D6

array([6, list([1, 0, 0, 1]), 3, 552.0, 1.0], dtype=object)

This divisors has the following properties: It corresponds to a dP$_0=\mathbb{P}^2$ surface with Hodge numbers

$h^{\bullet}(D_6) =\lbrace h^{0,0},h^{0,1}, h^{0,2},h^{1,1}\rbrace= \lbrace 1,0,0,1\rbrace$

The Euler number is given by

$\chi(D_6)=3$

while the Euler number for the class $4\cdot D_6$ is

$\chi(4\cdot D_6)=552$ .

Lastly, the arithmetic genus is determined as

$\chi_0(D_6)=1$ .


We can check whether it is a proper del Pezzo surface dP$_0$ inside the CY and whether it satisfies the diagonality condition by looking at **'DPEZ'**:

In [30]:
ptr["DPEZ"][T]

array([[6, 0, 1]])

We see that $D_6$ is a proper dP$_0$ and is diagonal. The latter is of course trivial for a dP$_0=\mathbb{P}^2$, but serves here as a sanity check!

In **Orientifold** we collect the number of inequivalent orientifolds (at least concerning the weight system). The orientifold data for one such configuration is encoded in another dictionary with the following keys:

In [10]:
O=1
Orientifold=Orientifolds[O]
Orientifold.keys()

dict_keys(['OIDA', 'OIDP', 'OIDTR', 'COORD', 'SING', 'CYEQ', 'OPC', 'OP', 'D3OP', 'D3SO8', 'D3W', 'H12PM'])

Here, the keys specify the folloing data

| Key | Quantity | Type | Length |
| --- | --- | --- | --- |
| **'OIDA'** | Orientifold ID total | scalar | 0 |
| **'OIDP'** | Orientifold ID polytope | scalar | 0 |
| **'OIDTR'** | Orientifold ID triangulation | scalar | 0 |
| **'COORD'** | Equivalent coordinates for reflection | list | - |
| **'SING'** | Singular | scalar | 0 |
| **'CYEQ'** | Invariant CY equation | array | - |
| **'OPC'** | Count of fixed point loci | list | 6 |
| **'OP'** | O$p_i$-planes $[[p_1,\sharp \,\text{planes},[z_k,z_l,\ldots]],\ldots]$ for fixed point loci $z_k=z_l=\ldots=0$ | array | -- |
| **'D3OP'** | D3-charge contribution from O$p$-planes $[Q_{O3},Q_{O7},Q_{O3}+Q_{O7}]$ | list | 3 |
| **'D3SO8'** | Total D3-charge for SO(8) stacks | scalar | 0 |
| **'D3W'** | Total D3-charge for Whitney branes | scalar | 0 |
| **'H12PM'** | Hodge numbers $h^{1,2}_{\pm}$ | list | 2 |

Let us look at the example used above:

In [11]:
Orientifold["OP"]

array([[7, 1, array([1])],
       [3, 3, array([2, 3, 5])],
       [3, 1, array([2, 3, 6])]], dtype=object)

This implies that we have:

* An O7-plane on $D_1$
* An O3-plane wrapping the locus $z_2=z_3=z_6=0$
* Three O3-planes wrapping the locus $z_2=z_3=z_5=0$

